
### 1. Read & clean up data

Datasets is open-source and were taken from https://huggingface.co/datasets/arkodeep/jobdata (english vacancies) and https://www.kaggle.com/datasets/vyacheslavpanteleev1/hhru-it-vacancies-from-20211025-to-20211202 (russian vacancies)


#### 1. read

We do not need all the data in the tables, so will throw out some of the columns


In [6]:
import pandas as pd

data_en = pd.read_csv('vacancies-skills_datasets/jobs_all.csv')
data_ru = pd.read_csv('vacancies-skills_datasets/IT_vacancies_full.csv')

In [7]:
data_en.columns

Index(['job_title_short', 'job_title', 'job_location', 'job_via',
       'job_schedule_type', 'job_work_from_home', 'search_location',
       'job_posted_date', 'job_no_degree_mention', 'job_health_insurance',
       'job_country', 'salary_rate', 'salary_year_avg', 'salary_hour_avg',
       'company_name', 'job_skills', 'job_type_skills', 'key_id'],
      dtype='object')

In [8]:
data_ru.columns

Index(['Ids', 'Employer', 'Name', 'Salary', 'From', 'To', 'Experience',
       'Schedule', 'Keys', 'Description', 'Area', 'Professional roles',
       'Specializations', 'Profarea names', 'Published at'],
      dtype='object')

In [9]:
data_en = data_en[['job_title_short','job_title','job_skills', 'job_type_skills']]
data_ru = data_ru[['Name','Specializations', 'Keys']]

In [10]:
data_en.head()

,job_title_short,job_title,job_skills,job_type_skills
0,Senior Data Engineer,Senior Clinical Data Engineer / Principal Clin...,NaN,NaN
1,Data Analyst,Data Analyst,"['r', 'python', 'sql', 'nosql', 'power bi', 't...","{'analyst_tools': ['power bi', 'tableau'], 'pr..."
2,Data Engineer,"Data Engineer/Scientist/Analyst, Mid or Senior...","['python', 'sql', 'c#', 'azure', 'airflow', 'd...","{'analyst_tools': ['dax'], 'cloud': ['azure'],..."
3,Data Engineer,LEAD ENGINEER - PRINCIPAL ANALYST - PRINCIPAL ...,"['python', 'c++', 'java', 'matlab', 'aws', 'te...","{'cloud': ['aws'], 'libraries': ['tensorflow',..."
4,Data Engineer,Data Engineer- Sr Jobs,"['bash', 'python', 'oracle', 'aws', 'ansible',...","{'cloud': ['oracle', 'aws'], 'other': ['ansibl..."


In [11]:
data_ru.head()

,Name,Specializations,Keys
0,Golang Developer (Кипр),"['Программирование, Разработка']","['Docker', 'Golang', 'Redis', 'Английский язык..."
1,Е-mail маркетолог,['Маркетинг'],"['Грамотность', 'Написание текстов', 'Грамотна..."
2,Оператор call-центра (удаленно),"['Маркетинг', 'Продажи по телефону, Телемаркет...","['Клиентоориентированность', 'Ориентация на ре..."
3,Ведущий SMM специалист,"['Управление маркетингом', 'PR, Маркетинговые ...","['Продвижение бренда', 'Креативность', 'Adobe ..."
4,UX/UI Designer,"['Игровое ПО', 'Программирование, Разработка',...","['UI', 'UX', 'gamedev', 'game design', 'проект..."



#### 2. Clean up text

Transfer everything to the lower register, remove Nans

In [12]:
import re

data_en = data_en.dropna()
data_ru = data_ru.dropna()

def clean(text):
    try:
        text = text.lower() #lower case
        text = re.sub(r'[-/()]', ' ', text)
        return text
    except Exception as e:
        print(text)

for dataset in data_en, data_ru:
    for col in dataset:
        dataset[col] = dataset[col].apply(clean)



In [13]:
print(data_en.head())

  job_title_short                                          job_title  \
1    data analyst                                       data analyst   
2   data engineer  data engineer scientist analyst, mid or senior...   
3   data engineer  lead engineer   principal analyst   principal ...   
4   data engineer                             data engineer  sr jobs   
5   data engineer                                  gcp data engineer   

                                          job_skills  \
1  ['r', 'python', 'sql', 'nosql', 'power bi', 't...   
2  ['python', 'sql', 'c#', 'azure', 'airflow', 'd...   
3  ['python', 'c++', 'java', 'matlab', 'aws', 'te...   
4  ['bash', 'python', 'oracle', 'aws', 'ansible',...   
5                           ['python', 'sql', 'gcp']   

                                     job_type_skills  
1  {'analyst_tools': ['power bi', 'tableau'], 'pr...  
2  {'analyst_tools': ['dax'], 'cloud': ['azure'],...  
3  {'cloud': ['aws'], 'libraries': ['tensorflow',...  
4  {'cloud


#### 3. tokenize & lemmatize

Tried spacy, then stanza (take too long). Now trying spacy again :)


In [14]:
import spacy

nlp_en = spacy.load("en_core_web_sm")
nlp_ru = spacy.load("ru_core_news_sm")

In [15]:
# import stanza
#
# stanza.download('en')
# stanza.download('ru')
#
# nlp_en = stanza.Pipeline('en', processors='tokenize,mwt,pos,lemma', use_gpu=False)
# nlp_ru = stanza.Pipeline('ru', processors='tokenize,pos,lemma', use_gpu=False)


In [16]:
def lemmatize_bulk(texts, nlp):
    docs = nlp.pipe(texts, batch_size=1000, disable=['ner'])
    return [" ".join([token.lemma_ for token in doc]) for doc in docs]

def lemmatize_list_bulk(items, nlp):
    return lemmatize_bulk(items, nlp)

def process_df(pdf, nlp, title_field='job_title', skills_field='job_skills'):
    pdf = pdf.copy()
    print('starting lemmatization of job titles')
    pdf[f'{title_field}_lemma'] = lemmatize_bulk(pdf[title_field].tolist(), nlp)

    print('starting lemmatization of skills')
    skills = [item for sublist in pdf[skills_field] for item in sublist]
    lemmatized_skills = lemmatize_bulk(skills, nlp)

    idx = 0
    lemmatized_skill_groups = []
    for skills in pdf[skills_field]:
        length = len(skills)
        lemmatized_skill_groups.append(lemmatized_skills[idx:idx + length])
        idx += length

    pdf[f'{skills_field}_lemma'] = lemmatized_skill_groups

    return pdf


In [ ]:
data_en_lemmatized = process_df(data_en, nlp_en)
data_en_lemmatized.head()

starting lemmatization of job titles


In [ ]:
data_ru_lemmatized = process_df(data_ru, nlp_ru, title_field='Name', skills_field='Keys')
data_ru_lemmatized.head()